In [14]:
#import necessary libraries
import json
import pickle
import nltk
import random
import numpy as np
from transformers import pipeline, PegasusForConditionalGeneration, PegasusTokenizer
from transformers import AutoTokenizer

In [15]:
import speech_recognition as sr
import pyautogui
import time
import threading
import datetime
from nltk.stem import WordNetLemmatizer
from keras.models import load_model
lemmatizer = WordNetLemmatizer()

In [16]:
intents = json.loads(open("intents.json").read())
words = pickle.load(open('LemmatizedWords.pkl', 'rb'))
classes = pickle.load(open('categories.pkl', 'rb'))
model = load_model('myBot.h5')

In [17]:
def Sentence_cleaner(stnc):
    w_sentence = nltk.word_tokenize(stnc)
    result= []
    for word in w_sentence:
        result.append(lemmatizer.lemmatize(word))
    return result

In [18]:
def bag_words(stnc):
    w_sentence = Sentence_cleaner(stnc)
    bag = [0] * len(words)
    for w in w_sentence:
        for i, Words in enumerate(words):
            if Words == w:
                bag[i] = 1
    res = np.array(bag)
    return res

In [19]:
def class_predictor(stnc):
    ERROR_THRESHOLD = 0.25
    res_list = []
    bag = bag_words(stnc)
    res = model.predict(np.array([bag]))[0]
    res = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    res.sort(key=lambda x: x[1], reverse=True)
    for r in res:
        res_list.append({'intent': classes [r[0]],'probability': str(r[1])})
    return res_list

In [20]:
hover_stop_flag = False

In [21]:
def hover(direction):
    while not hover_stop_flag:
        if direction == "up":
            pyautogui.move(0, -10)
        elif direction == "down":
            pyautogui.move(0, 10)
        elif direction == "left":
            pyautogui.move(-10, 0)
        elif direction == "right":
            pyautogui.move(10, 0)
        time.sleep(0.1)

In [22]:

def stop_hover():
    global hover_stop_flag
    hover_stop_flag = True

In [23]:
def respond(intents_list, intents_json):
    global hover_stop_flag
    if not intents_list:
        print("No intent Found!")
        return
    tag = intents_list[0]['intent']
    list_json_intent = intents_json['intents']
    for i in list_json_intent:
        if i['tag'] == tag:
            if tag == "current_time":
                exactTime = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
                randRespond = random.choice(i['responses'])
                result = randRespond.format(time_variable =exactTime)   
            elif tag == "Summarization":
                result = random.choice(i['responses'])
                print(result)
                summarizer = input(">  ")
                #Using pegasus-xsum model to generate new model
                tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
                #Model
                model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
                tokens = tokenizer(summarizer, truncation=True, padding="longest", return_tensors="pt")
                summarizeText = model.generate(**tokens)
                res = tokenizer.decode(summarizeText[0], skip_special_tokens=True)
                return res
            elif tag == "mouseclick":
                pyautogui.click()
                result = random.choice(i['responses'])
            elif tag in {"hoverUP", "hoverDown", "hoverLeft", "hoverRight"}:
                hover_stop_flag = False
                direction = tag.replace("hover", "").lower()
                hover_thread = threading.Thread(target=lambda: hover(direction))
                result = random.choice(i['responses'])
                hover_thread.start()
            elif tag == "stop":
                result = random.choice(i['responses'])
                stop_hover()
            elif tag == "goodbye":           
                result = random.choice(i['responses'])  
            else:
                result = random.choice(i['responses'])
            break
    return result

In [24]:
while True:
    textInput = input("")
    if textInput == "==":
        break
    ints = class_predictor(textInput)
    print("Intents:", ints)
    res = respond(ints, intents)
    print(res)

1/1 [==============================] - 0s 151ms/step
Intents: [{'intent': 'greeting', 'probability': '0.7764788'}]
greetings
1/1 [==============================] - 0s 31ms/step
Intents: [{'intent': 'greeting', 'probability': '0.7764788'}]
Need something?
1/1 [==============================] - 0s 24ms/step
Intents: [{'intent': 'greeting', 'probability': '0.7764788'}]
Hello!
1/1 [==============================] - 0s 38ms/step
Intents: [{'intent': 'current_time', 'probability': '0.9999654'}]
16/01/2024 05:03:47
1/1 [==============================] - 0s 24ms/step
Intents: [{'intent': 'Summarization', 'probability': '0.9999367'}]
Sure! what do you wanna summarize?


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


A chatbot is a computer program that simulates and processes human conversation (either written or spoken), allowing humans to interact with digital devices as if they were communicating with a real person.


In [ ]:
"""
At the most basic level, a chatbot is a computer program that simulates and processes human conversation (either written or spoken), allowing humans to interact with digital devices as if they were communicating with a real person. Chatbots can be as simple as rudimentary programs that answer a simple query with a single-line response, or as sophisticated as digital assistants that learn and evolve to deliver increasing levels of personalization as they gather and process information.


You’ve probably interacted with a chatbot whether you know it or not. For example, you’re at your computer researching a product, and a window pops up on your screen asking if you need help. Or perhaps you’re on your way to a concert and you use your smartphone to request a ride via chat. Or you might have used voice commands to order a coffee from your neighborhood café and received a response telling you when your order will be ready and what it will cost. These are all examples of scenarios in which you could be encountering a chatbot.

"""
#https://www.oracle.com/chatbots/what-is-a-chatbot/#:~:text=Chatbots%20are%20conversational%20tools%20that,cannot%20be%20replicated%20by%20machines.


In [ ]:
recognizer = sr.Recognizer()
# for mic 
while True:
    
    with sr.Microphone() as source:
        print("Listening")
        recognizer.adjust_for_ambient_noise(source)
        audio_data = recognizer.listen(source)

    try:
        text_audio = recognizer.recognize_google(audio_data, language="en-US")
        print("Recognized Text:", text_audio)

        ints = class_predictor(text_audio)
        print("Intents:", ints)

        res = respond(ints, intents)
        print(res)

    except sr.UnknownValueError:
        print("Speech Recognition could not understand audio")
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
